# Sweeps - Capacitance matrix

### Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

In [4]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs.

In [5]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '5.7 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2 Metal passes information to the simulator "q3d" to extract the capacitance matrix.


In [6]:
c1 = LOManalysis(design, "q3d")

Prepare data to pass as arguments for method run_sweep().

In [7]:
render_design_argument_qcomps = ['Q1']

render_design_argument_endcaps = [('Q1', 'readout'), ('Q1', 'bus1'),('Q1', 'bus2')]


In [8]:
# To identify the agruments that you can change.
# They will change based on the simulation software used.
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 15,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [9]:
# For the simulation software, if you don't want to use the default values, 
# you can update them as seen below.

# If a setup named "sweeper_q3d_setup" exists in the project, it will be deleted, 
# and a new setup will be added.
c1.sim.setup.name = "sweeper_q3d_setup"

c1.sim.setup.freq_ghz = 5.6
c1.sim.setup.max_passes = 9
c1.sim.setup.min_passes = 2
c1.sim.setup.percent_error = 0.45



We will look at modifying the pad_gap of qubit 1, to see how it impacts the anharmonicity of the qubit.

The steps will be;
- Connect to Ansys Q3D.
- Rebuild QComponents in Metal.
- Render QComponents within Q3D and setup the simulation.
- Delete/Clear the Q3D between each simulation.
- Using the capacitance matrices, LOM for each value in option_sweep is found.

#### Returns a dict and return code.  If the return code is zero, there were no errors detected.  
#### The dict has:  key = each value used to sweep, value = capacitance matrix

#### This could take minutes or hours based on the complexity of the design.



In [10]:
sweep_data, return_code = c1.run_sweep(q1.name,
                                       'pad_gap',
                                       ['20um', '30um', '40um'],
                                       render_design_argument_qcomps,
                                       render_design_argument_endcaps,
                                       design_name="GetCapacitance",
                                       box_plus_buffer=True)

INFO 03:48PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:48PM [load_ansys_project]: 	Opened Ansys App
INFO 03:48PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 03:48PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/askev/OneDrive/文件/Ansoft/
	Project:   Project3
INFO 03:48PM [connect_design]: 	Opened active design
	Design:    GetEigenModeSolution_hfss [Solution type: Eigenmode]
INFO 03:48PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:48PM [connect]: 	Connected to project "Project3" and design "GetEigenModeSolution_hfss" 😀 

INFO 03:48PM [connect_design]: 	Opened active design
	Design:    GetCapacitance_q3d [Solution type: Q3D]
WARNING 03:48PM [connect_setup]: 	No design setup detected.
WARNING 03:48PM [connect_setup]: 	Creating Q3D default setup.
INFO 03:48PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 03:48PM [get_setup]: 	Opened setup `sweeper_q3d_setup`  (<class 'p

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.247412 [GHz]
EC 282.297483 [MHz]
EJ 13.616300 [GHz]
alpha -325.270823 [MHz]
dispersion 15.110759 [KHz]
Lq 11.995161 [nH]
Cq 68.616368 [fF]
T1 55.257386 [us]

**Coupling Properties**

tCqbus1 7.541487 [fF]
gbus1_in_MHz 108.609156 [MHz]
χ_bus1 -2.159784 [MHz]
1/T1bus1 1973.934040 [Hz]
T1bus1 80.628299 [us]

tCqbus2 -6.652598 [fF]
gbus2_in_MHz -82.320882 [MHz]
χ_bus2 -5.470586 [MHz]
1/T1bus2 625.030509 [Hz]
T1bus2 254.635479 [us]

tCqbus3 5.476783 [fF]
gbus3_in_MHz 69.986339 [MHz]
χ_bus3 -2.642687 [MHz]
1/T1bus3 281.282847 [Hz]
T1bus3 565.818161 [us]
Bus-Bus Couplings
gbus1_2 7.514495 [MHz]
gbus1_3 10.425045 [MHz]
gbus2_3 5.686929 [MHz]


INFO 03:48PM [connect_design]: 	Opened active design
	Design:    GetCapacitance_q3d [Solution type: Q3D]
INFO 03:48PM [get_setup]: 	Opened setup `sweeper_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 03:48PM [analyze]: Analyzing setup sweeper_q3d_setup
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmp69e2t4nc.txt, C, , sweeper_q3d_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmppahhiq8_.txt, C, , sweeper_q3d_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmpfk51bkuc.txt, C, , sweeper_q3d_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 2, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmp71adc508.txt, C, ,

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.461668 [GHz]
EC 307.453057 [MHz]
EJ 13.616300 [GHz]
alpha -357.099183 [MHz]
dispersion 33.328972 [KHz]
Lq 11.995161 [nH]
Cq 63.002230 [fF]
T1 31.296952 [us]

**Coupling Properties**

tCqbus1 7.549943 [fF]
gbus1_in_MHz 116.235853 [MHz]
χ_bus1 -3.373308 [MHz]
1/T1bus1 3070.694236 [Hz]
T1bus1 51.830280 [us]

tCqbus2 -6.638235 [fF]
gbus2_in_MHz -87.812044 [MHz]
χ_bus2 -11.467976 [MHz]
1/T1bus2 1450.021950 [Hz]
T1bus2 109.760368 [us]

tCqbus3 5.506337 [fF]
gbus3_in_MHz 75.219769 [MHz]
χ_bus3 -5.026928 [MHz]
1/T1bus3 564.601440 [Hz]
T1bus3 281.889014 [us]
Bus-Bus Couplings
gbus1_2 7.286932 [MHz]
gbus1_3 10.181252 [MHz]
gbus2_3 5.497324 [MHz]


INFO 03:49PM [connect_design]: 	Opened active design
	Design:    GetCapacitance_q3d [Solution type: Q3D]
INFO 03:49PM [get_setup]: 	Opened setup `sweeper_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 03:49PM [analyze]: Analyzing setup sweeper_q3d_setup
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmpboiqwk9r.txt, C, , sweeper_q3d_setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmplefwup8m.txt, C, , sweeper_q3d_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmpdtw6jl1a.txt, C, , sweeper_q3d_setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 2, False
INFO 03:49PM [get_matrix]: Exporting matrix data to (C:\Users\askev\AppData\Local\Temp\tmpujn3iiti.txt, C, ,

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.602839 [GHz]
EC 324.703464 [MHz]
EJ 13.616300 [GHz]
alpha -379.187685 [MHz]
dispersion 54.270365 [KHz]
Lq 11.995161 [nH]
Cq 59.655132 [fF]
T1 19.186020 [us]

**Coupling Properties**

tCqbus1 7.642104 [fF]
gbus1_in_MHz 122.796596 [MHz]
χ_bus1 -4.681908 [MHz]
1/T1bus1 4274.812004 [Hz]
T1bus1 37.230864 [us]

tCqbus2 -6.692781 [fF]
gbus2_in_MHz -92.403963 [MHz]
χ_bus2 -21.050791 [MHz]
1/T1bus2 3029.326829 [Hz]
T1bus2 52.538056 [us]

tCqbus3 5.577996 [fF]
gbus3_in_MHz 79.529733 [MHz]
χ_bus3 -8.262669 [MHz]
1/T1bus3 991.221218 [Hz]
T1bus3 160.564504 [us]
Bus-Bus Couplings
gbus1_2 7.165878 [MHz]
gbus1_3 10.047044 [MHz]
gbus2_3 5.368402 [MHz]


In [11]:
from pandas import DataFrame

ec_val = []
for opt_val in sweep_data.keys():
    ec_val.append([opt_val,sweep_data[opt_val]['variables']['lumped_oscillator']['EC']])

df=DataFrame(ec_val,columns = ['Sweep Value', 'Ec'])
df

,Sweep Value,Ec
0,20um,282.297483
1,30um,307.453057
2,40um,324.703464


We can grab specific values from the results as seen below;

In [12]:
sweep_data.keys()

dict_keys(['20um', '30um', '40um'])

In [13]:
# For each value of option, there is a set of data.
sweep_data['20um'].keys()

dict_keys(['option_name', 'variables', 'sim_variables'])

In [14]:
sweep_data['20um']['variables'].keys()

dict_keys(['lumped_oscillator', 'lumped_oscillator_all'])

In [15]:
sweep_data['20um']['sim_variables'].keys()

dict_keys(['sim_setup_name', 'cap_matrix', 'units', 'cap_all_passes'])

A boolean flag indicating whether a sweep has converged can be obtained from the `is_converged` key within the `sim_variables`. For instance, the following code retrieves the convergence result for the sweep parameter, 20um. The convergence is extracted from the Ansys Q3D renderer.

In [16]:
sweep_data['20um']['sim_variables']['is_converged']

{}

In [17]:
if return_code ==0:
    print(f'{sweep_data.keys()} \n')
    for key in sweep_data.keys():
        print(f'\nkey={key}')
        
        option_name = sweep_data[key]['option_name']
        print(f'option_name[\'{key}\'][\'option_name\']={option_name}')
        
    
        variables = sweep_data[key]['variables']
        sim_variables = sweep_data[key]['sim_variables']
        
        print(f'variables={variables}')
        print(f'sim_variables={sim_variables}')
        

        

dict_keys(['20um', '30um', '40um']) 


key=20um
option_name['20um']['option_name']=pad_gap
variables={'lumped_oscillator': {'fQ': 5.247411523116215, 'EC': 282.297482844184, 'EJ': 13.616300010297985, 'alpha': -325.2708230921257, 'dispersion': 15.110758714675903, 'gbus': array([108.60915642, -82.32088157,  69.98633941]), 'chi_in_MHz': array([-2.15978423, -5.47058576, -2.64268722])}, 'lumped_oscillator_all':          fQ          EC       EJ       alpha dispersion  \
1  5.694788  336.232061  13.6163 -394.070196  73.574683   
2  5.571993  320.887984  13.6163 -374.283557  48.891198   
3  5.451287  306.205866  13.6163 -355.510519  32.122595   
4  5.384935   298.30272  13.6163 -345.469437  25.292174   
5  5.338877  292.886292  13.6163 -338.613528  21.348784   
6  5.299214  288.267285  13.6163 -332.783452   18.40506   
7  5.283239  286.418644  13.6163 -330.454353  17.326278   
8  5.264375  284.244544  13.6163 -327.718307  16.125855   
9  5.247412  282.297483  13.6163 -325.270823  15.110759   



# READ THIS BEFORE running the cell.
This cell is to demonstrate that if you have already executed c1.sim.run(), you don't have to set up 
the environment again for c1.run_sweep().  In another words, if you don't pass updated arguments to
c1.run_sweep(), then c1.run_sweep() looks for the previous desgin arguments. 

If you pass anything more than these three arguments: qcomp_name, option_name, option_sweep ..... 
Then NOTHING will be used from previous run.  
```
c1.sim.solution_order = 'Medium'
c1.sim.auto_increase_solution_order = 'False'


c1.sim.run(components=render_design_argument_qcomps,
           open_terminations=render_design_argument_endcaps)
```

Because c1.sim.setup.run has the information from last run, this is OK.

```
sweep_data, return_code = c1.run_sweep(q1.name, 
                                        'pad_gap', 
                                        ['20um', '30um', '40um'])
```

In [18]:
c1.sim.close()

In [19]:
# Uncomment next line if you would like to close the gui
gui.main_window.close()

True